In [46]:
import autograd
import autograd.numpy as np

import scipy.integrate
solve_ivp = scipy.integrate.solve_ivp
#from utils import to_pickle, from_pickle
#import gym
#import myenv

def hamiltonian_fn(coords):
    q, p = np.split(coords,2)
    # pendulum hamiltonian conosistent with openAI gym Pendulum-v0
    H = 5*(1-np.cos(q)) + 1.5 * p**2 
    return H

def dynamics_fn(t, coords, u=0):
    dcoords = autograd.grad(hamiltonian_fn)(coords)
    dqdt, dpdt = np.split(dcoords,2)
    S = np.concatenate([dpdt, -dqdt + u], axis=-1)
    return S

def get_trajectory(timesteps=20, radius=None, y0=None, noise_std=0.1, u=0.0, rad=False, **kwargs):
    t_eval = np.linspace(1, timesteps, timesteps) * 0.05
    t_span = [0.05, timesteps*0.05]

    # get initial state
    if rad:
        if y0 is None:
            y0 = np.random.rand(2)*2.-1
        if radius is None:
            radius = np.random.rand() + 1.3 # sample a range of radii
        y0 = y0 / np.sqrt((y0**2).sum()) * radius ## set the appropriate radius
    else:
        if y0 is None:
            y0 = np.random.rand(2) * 3 * np.pi - np.pi

    spring_ivp = solve_ivp(lambda t, y: dynamics_fn(t, y, u), t_span=t_span, y0=y0, t_eval=t_eval, rtol=1e-10, **kwargs)
    q, p = spring_ivp['y'][0], spring_ivp['y'][1]

    # add noise
    q += np.random.randn(*q.shape)*noise_std
    p += np.random.randn(*p.shape)*noise_std

    return q, p, t_eval


def get_dataset(seed=0, samples=50, test_split=0.5, gym=False, save_dir=None, us=[0], rad=False, timesteps=20, **kwargs):
    data = {}
   
    xs_force = []
    for u in us:
        xs = []
        np.random.seed(seed)
        for _ in range(samples):
            q, p, t = get_trajectory(noise_std=0.0, u=u, rad=rad, timesteps=timesteps, **kwargs)
            xs.append(np.stack((q, p, np.ones_like(q)*u), axis=1)) # (45, 3) last dimension is u
        xs_force.append(np.stack(xs, axis=1)) # fit Neural ODE format (45, 50, 3)
        
    data['x'] = np.stack(xs_force, axis=0) # (3, 45, 50, 3)

    # make a train/test split
    split_ix = int(samples * test_split)
    split_data = {}
    split_data['x'], split_data['test_x'] = data['x'][:,:,:split_ix,:], data['x'][:,:,split_ix:,:]

    data = split_data
    data['t'] = t
    
    return data

def arrange_data(x, t, num_points=2):
    '''Arrange data to feed into neural ODE in small chunks'''
    assert num_points>=2 and num_points<=len(t)
    x_stack = []
    for i in range(num_points):
        print('i',i)
        if i < num_points-1:
            print(x[:, i:-num_points+i+1,:,:].shape)
            x_stack.append(x[:, i:-num_points+i+1,:,:])
        else:
            x_stack.append(x[:, i:,:,:])
    x_stack = np.stack(x_stack, axis=1)
    print(x_stack)
    x_stack = np.reshape(x_stack, 
                (x.shape[0], num_points, -1, x.shape[3]))
    t_eval = t[0:num_points]
    return x_stack, t_eval

In [37]:
x.shape

(1, 20, 25, 3)

In [4]:
get_trajectory(timesteps=20, radius=None, y0=None, noise_std=0.1, u=0.0, rad=False)

(array([ 3.37446548,  3.49376999,  4.27011998,  4.63656485,  5.297462  ,
         5.77496744,  6.16167111,  6.98998215,  7.59774587,  7.90518921,
         8.58498537,  9.03622721,  9.36050043,  9.82759589, 10.38639233,
        10.88314451, 11.3942269 , 11.9956593 , 12.54991904, 13.12167598]),
 array([2.84044524, 3.12848425, 3.05865623, 3.33257518, 3.52472734,
        3.89882438, 4.03153822, 3.81593044, 3.68609135, 3.36957709,
        3.18892415, 3.17207499, 2.8840481 , 2.95691085, 3.07338303,
        3.36329477, 3.81314246, 3.80532995, 3.98262358, 3.77952608]),
 array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
        0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]))

In [25]:
data = get_dataset(seed=0, samples=50, test_split=0.5, gym=False, save_dir=None, us=[0], rad=False, timesteps=20)

In [26]:
data

{'x': array([[[[ 2.03085276,  3.59890832,  0.        ],
          [ 2.2323853 ,  0.99212922,  0.        ],
          [ 2.43827109,  2.2517545 ,  0.        ],
          ...,
          [-2.95816322,  0.62098967,  0.        ],
          [ 1.19262907,  2.26455185,  0.        ],
          [ 4.60466919,  3.46073407,  0.        ]],
 
         [[ 2.55573522,  3.41389476,  0.        ],
          [ 2.3669783 ,  0.8061779 ,  0.        ],
          [ 2.76559342,  2.12420691,  0.        ],
          ...,
          [-2.86101339,  0.67827505,  0.        ],
          [ 1.51428415,  2.02132206,  0.        ],
          [ 5.14233842,  3.70465457,  0.        ]],
 
         [[ 3.06028331,  3.3331398 ,  0.        ],
          [ 2.47531553,  0.64171505,  0.        ],
          [ 3.07922279,  2.07023536,  0.        ],
          ...,
          [-2.75345759,  0.76003486,  0.        ],
          [ 1.79879153,  1.77314771,  0.        ],
          [ 5.71318611,  3.89119528,  0.        ]],
 
         ...,
 
       

In [27]:
data['x'].shape

(1, 20, 25, 3)

In [36]:
20*25

500

In [48]:
x = data['x']

In [72]:
num_points=2

In [73]:
x.shape

(1, 20, 25, 3)

In [74]:
assert num_points>=2 and num_points<=len(t)
x_stack = []
for i in range(num_points):
    print('i',i)
    if i < num_points-1:
        print(x[:, i:-num_points+i+1,:,:].shape)
        print('number',-num_points+i+1)
        x_stack.append(x[:, i:-num_points+i+1,:,:])
    else:
        x_stack.append(x[:, i:,:,:])



i 0
(1, 19, 25, 3)
number -1
i 1


In [58]:
x_stack[1].shape

(1, 19, 25, 3)

In [61]:
x_stack[0].shape

(1, 19, 25, 3)

In [62]:
x_stack = np.stack(x_stack, axis=1)

In [63]:
x_stack.shape

(1, 2, 19, 25, 3)

In [ ]:

print(x_stack)
x_stack = np.reshape(x_stack, 
            (x.shape[0], num_points, -1, x.shape[3]))
t_eval = t[0:num_points]

In [29]:
t = data['t']

In [30]:
x

array([[[[ 2.03085276,  3.59890832,  0.        ],
         [ 2.2323853 ,  0.99212922,  0.        ],
         [ 2.43827109,  2.2517545 ,  0.        ],
         ...,
         [-2.95816322,  0.62098967,  0.        ],
         [ 1.19262907,  2.26455185,  0.        ],
         [ 4.60466919,  3.46073407,  0.        ]],

        [[ 2.55573522,  3.41389476,  0.        ],
         [ 2.3669783 ,  0.8061779 ,  0.        ],
         [ 2.76559342,  2.12420691,  0.        ],
         ...,
         [-2.86101339,  0.67827505,  0.        ],
         [ 1.51428415,  2.02132206,  0.        ],
         [ 5.14233842,  3.70465457,  0.        ]],

        [[ 3.06028331,  3.3331398 ,  0.        ],
         [ 2.47531553,  0.64171505,  0.        ],
         [ 3.07922279,  2.07023536,  0.        ],
         ...,
         [-2.75345759,  0.76003486,  0.        ],
         [ 1.79879153,  1.77314771,  0.        ],
         [ 5.71318611,  3.89119528,  0.        ]],

        ...,

        [[11.40668341,  3.97042248,  0

In [47]:
x_arranged = arrange_data(x, t, num_points=2)

i 0
(1, 19, 25, 3)
i 1


In [41]:
x_arranged[0].shape

(1, 2, 475, 3)

In [45]:
x_arranged[0][:,:,4,:]

array([[[2.40316551, 4.69085451, 0.        ],
        [3.09782728, 4.59772455, 0.        ]]])

In [33]:
x_arranged[1]

array([0.05, 0.1 ])

In [34]:
x_arranged[1].shape

(2,)

In [35]:
x_arranged[2].shape

IndexError: tuple index out of range

In [1]:
import torch

In [6]:
a = torch.tensor([1,2,3,4])
a[[1,2]]

tensor([2, 3])